# Download tool for the UAVSAR RSLC's dealing with dithered data

In [1]:
import os, shutil
import subprocess as sp

In [2]:
#indicate flightline details
CAMPAIGN='NISARP' #or 'NISARA' for AM and 'NISARP' for PM
SITE='25006'
PROD='138' #129 = global 20 & 5 MHz, 138 = US Ag 40 & 5 MHz
PROD2='CG' #CX = normal, CG = dithered-gaps, CD = dithered-no-gaps
URLROOT = 'https://uavsar.asf.alaska.edu'
# URLROOT = 'https://uavsar.jpl.nasa.gov/cgi-bin/data.pl' #this doesnt work
if CAMPAIGN=='NISARA':
    #download root web address for the AM datasets
    DOWNLOADROOT='https://uavsar.jpl.nasa.gov/Release2v' #AM
    releaseroot = 'http://downloaduav.jpl.nasa.gov/Release2v/' #AM
elif CAMPAIGN=='NISARP':
    #download root web address for the PM datasets
    DOWNLOADROOT='https://uavsar.jpl.nasa.gov/Release2m' #PM
    releaseroot = 'http://downloaduav.jpl.nasa.gov/Release2m/' #PM

basedir= os.getcwd()
workingdir=os.path.join(basedir, SITE, PROD, PROD2)
os.makedirs(workingdir,exist_ok='True')
if not os.path.exists(workingdir):
    os.mkdir(workingdir)
UAVSARTOOL= os.path.join(basedir, 'seppo_uavsar_download.py')

In [3]:
# Only run if you didn't install the download tool yet

# HOWTO:
# 1) download tool from here: https://raw.githubusercontent.com/EarthBigData/openSAR/master/code/seppo/seppo_uavsar_download.py
# 2) chmod +x seppo_uavsar_download.py
# 3) may need to change the shebang in the seppo tool from python to python3

# Get the open sourced UAVSAR download tool to find data take ids:
# cmd='wget -c -O {UAVSARTOOL} https://raw.githubusercontent.com/EarthBigData/openSAR/master/code/seppo/seppo_uavsar_download.py; chmod 755 {UAVSARTOOL}'.format(UAVSARTOOL=UAVSARTOOL)
# os.system(cmd)

In [4]:
# If you never built the index (that is, there is no datatake_27900 or so file listing all the flights)
# run this command with -the -r switch; may take some time
#BUILDINDEX='-r'
# Otherwise uncomment below

BUILDINDEX=''
DATATAKES=os.path.join(workingdir, 'datatakes_'+SITE) #the datatake text file
cmd=UAVSARTOOL+' {BUILDINDEX} --urlroot {URLROOT} --site {SITE} --campaign {CAMPAIGN} -ldt > {DATATAKES}'.format(BUILDINDEX=BUILDINDEX,URLROOT=URLROOT,SITE=SITE,CAMPAIGN=CAMPAIGN,DATATAKES=DATATAKES)
print(cmd)
os.system(cmd)

/data1/uavsar/uavsar_h5slc2mlc_rev/seppo_uavsar_download.py  --urlroot https://uavsar.asf.alaska.edu --site 25006 --campaign NISARP -ldt > /data1/uavsar/uavsar_h5slc2mlc_rev/25006/138/CG/datatakes_25006


256

In [5]:
#------------------------Can be commented after data folder operations were performed once (download, reprojections)
#READ IN THE DATA TAKES FILE
datatakes = []
with open(DATATAKES,"r") as f:
    for num, val in enumerate(f.readlines()):
        if val.startswith('UA_NISAR') and not val.endswith('04\n'):
            #condition needed because
            #1) sometimes the first line in this file does not contain any UAVSAR related information, i.e. when BUILDINDEX -r
            #2) ASF lists a v4 product that doesn't actually exist. Valid datasets can be found on the NASA JPL website shown above. 
            datatakes.append(val)

# fixes the formatting of the above list 
dt_tmp=[x.lstrip('UA_').rstrip('\n') for x in datatakes]
print('All available data:')
print('\n'.join(dt_tmp))

# builds a dictionary structure with the datatake id and list of the available versions
dt={}
for i in dt_tmp:
    tokens=i.split('_')
    dtroot='_'.join(tokens[:-1])
    if not dtroot in dt:
        dt[dtroot]=[]
    dt[dtroot].append(tokens[-1])
#print(dt)

# always choose the latest version, because it is the most error corrected/up-to-data data that should be used 
product_ids=[]
for i in dt:
    product_ids.append(i+'_'+dt[i][-1])
product_ids.sort()

print('Most updated data:')
print('\n'.join(product_ids))

All available data:
NISARP_25006_19040_009_190621_L090_CX_02
NISARP_25006_19049_013_190718_L090_CX_01
NISARP_25006_19049_013_190718_L090_CX_02
NISARP_25006_19052_011_190727_L090_CX_01
NISARP_25006_19054_005_190814_L090_CX_01
NISARP_25006_19071_007_191002_L090_CX_01
NISARP_25006_19078_007_191016_L090_CX_01
Most updated data:
NISARP_25006_19040_009_190621_L090_CX_02
NISARP_25006_19049_013_190718_L090_CX_02
NISARP_25006_19052_011_190727_L090_CX_01
NISARP_25006_19054_005_190814_L090_CX_01
NISARP_25006_19071_007_191002_L090_CX_01
NISARP_25006_19078_007_191016_L090_CX_01


### We may need to repair some data references

UA_NISARP_25006 campaign 
1. Versions need to be incremented by 1

In [6]:
#------------------------Can be commented after data folder operations were performed once (download, reprojections)
if SITE=='25006':
    for num, val in enumerate(product_ids):
        aa = val.split('CX_')
        vernum = int(aa[-1])+1
        product_ids[num] = aa[0]+'CX_'+'0'+str(vernum)
print(product_ids)

['NISARP_25006_19040_009_190621_L090_CX_03', 'NISARP_25006_19049_013_190718_L090_CX_03', 'NISARP_25006_19052_011_190727_L090_CX_02', 'NISARP_25006_19054_005_190814_L090_CX_02', 'NISARP_25006_19071_007_191002_L090_CX_02', 'NISARP_25006_19078_007_191016_L090_CX_02']


## Download the data takes
We construct the downloadlinks for the RSLC.

In [7]:
#------------------------Can be commented after data folder operations were performed once (download, reprojections)

#this function programmatically generates the download links of each RSLC file
#DOWNLOADROOT='https://uavsar.jpl.nasa.gov/Release2u'

def get_download_links(product_id,root=releaseroot):    
    dl={}
    dl['annA']    = root.rstrip('/')+'/'+product_id+'//'+product_id.replace('_CX_','_CX_'+PROD+'A_')+'.ann'
    dl['annB']    = root.rstrip('/')+'/'+product_id+'//'+product_id.replace('_CX_','_CX_'+PROD+'B_')+'.ann'
    dl['h5']     = root.rstrip('/')+'/'+product_id+'//'+product_id.replace('_CX_','_'+PROD2+'_'+PROD+'_')+'.h5'
    return dl

# build dictionary of links where the UAVSAR annotation file (ann), the radiometric and terrain correction factor (rtcf), the flat earth incidence angle (flatinc) and the HVHV GRD data can be downloaded
dl={}
for i in product_ids:
    dl[i]=get_download_links(i,root=DOWNLOADROOT)

In [8]:
dl

{'NISARP_25006_19040_009_190621_L090_CX_03': {'annA': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CX_138A_03.ann',
  'annB': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CX_138B_03.ann',
  'h5': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CG_138_03.h5'},
 'NISARP_25006_19049_013_190718_L090_CX_03': {'annA': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CX_138A_03.ann',
  'annB': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CX_138B_03.ann',
  'h5': 'https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CG_138_03.h5'},
 'NISARP_25006_19052_011_190727_L090_CX_02': {'annA': 'https

In [9]:
#------------------------Can be commented after data folder operations were performed once (download, reprojections)
os.chdir(workingdir)
failed=[]

for dt in dl:
    for i in dl[dt]:
        try:
            cmd='wget -c {}'.format(dl[dt][i])
            print(cmd)
            sp.check_call(cmd.split())
        except:
            failed.append(dl[dt][i])
            print('cannot  download',dl[dt][i])
if failed:
        print('Please manually edit the download list with an valid version number and rerun from cell "GET DOWNLOAD URLS"')

wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CX_138A_03.ann
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CX_138B_03.ann
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19040_009_190621_L090_CX_03//NISARP_25006_19040_009_190621_L090_CG_138_03.h5
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CX_138A_03.ann
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CX_138B_03.ann
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19049_013_190718_L090_CX_03//NISARP_25006_19049_013_190718_L090_CG_138_03.h5
wget -c https://uavsar.jpl.nasa.gov/Release2m/NISARP_25006_19052_011_190727_L090_CX_02//NISARP_25006_19052_011_190727_L090_CX_138A_02.ann
wget -c https://uavsar.jpl.nasa.gov/Re